In [1]:
!pip install xarray  rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 67.6 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
!apt-get install -y gdal-bin


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3-gdal python3-numpy
Suggested packages:
  libgdal-grass python-numpy-doc python3-pytest
The following NEW packages will be installed:
  gdal-bin python3-gdal python3-numpy
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 5,055 kB of archives.
After this operation, 25.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 python3-numpy amd64 1:1.21.5-1ubuntu22.04.1 [3,467 kB]
Get:2 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy/main amd64 python3-gdal amd64 3.6.4+dfsg-1~jammy0 [1,027 kB]
Get:3 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy/main amd64 gdal-bin amd64 3.6.4+dfsg-1~jammy0 [561 kB]
Fetched 5,055 kB in 2s (2,752 kB/s)
Selecting previously unselected package python3-numpy.
(Reading database ... 123635 files and direc

In [3]:
import ftplib
import os
import xarray as xr
import pandas as pd
import numpy as np
import rasterio
from rasterio.transform import from_bounds
from scipy.interpolate import griddata
import subprocess
from google.cloud import storage
import ee

In [4]:
ee.Authenticate()
# Initialize Earth Engine
ee.Initialize(project='pyregence-ee')

**Models**
* ACCESS1-0
* BCC-CSM1-1
* BCC-CSM1-1-M
* BNU-ESM
* CanESM2
* CCSM4
* CESM1-BGC
* CESM1-CAM5
* CMCC-CM
* CNRM-CM5
* CSIRO-Mk3-6-0
* FGOALS-G2
* FIO-ESM
* GFDL-CM3
* GFDL-ESM2G
* GFDL-ESM2M
* GISS-E2-H-CC
* GISS-E2-R
* GISS-E2-R-CC
* HadGEM2-AO
* HadGEM2-CC
* HadGEM2-ES
* INMCM4
* IPSL-CM5A-MR
* IPSL-CM5B-LR
* MIROC-ESM
* MIROC-ESM-CHEM
* MIROC5
* MPI-ESM-LR
* MPI-ESM-MR
* MRI-CGCM3
* NorESM1-M

**Variables**
- albedo
- baseflow
- del_SWE
- ET
- latent
- longwave_net
- pet_natveg
- precip
- Qair
- rainfall
- rel_Humid
- runoff
- sensible
- shortwave_in
- shortwave_net
- snow_melt
- snowfall
- soilMoist1
- soilMoist2
- soilMoist3
- sublimation_net
- SWE
- Tair
- Wdew
- windspeed

In [8]:
def download_model_files(model, year, variable, scenario="rcp85", local_destination=None):
    """
    Download a specific NetCDF file for a model and year from the FTP server.

    Parameters
    ----------
    model: str
        Model name (e.g., "CESM1-BGC", "CanESM2").
    year: int
        Year of the desired file.
    variable: str
        Variable to download (e.g., "ET").
    scenario: str
        Climate scenario for future projections (default: "rcp85"). "rcp45"
        Used for years after 2005.
    local_destination: str
        Destination folder to save files locally. Defaults to current working directory.

    Returns
    -------
    local_file_path: str
        Path to the downloaded file or None if the file was not found.
    """
    ftp_server = "gdo-dcp.ucllnl.org"
    base_path = "/pub/dcp/archive/cmip5/loca_hydro/LOCA_VIC_dpierce_2017-02-28"

    # Set local destination
    if local_destination is None:
        local_destination = os.getcwd()
    if not os.path.exists(local_destination):
        os.makedirs(local_destination)

    # Connect to FTP server
    ftp = ftplib.FTP(ftp_server)
    ftp.login()
    print(f"Connected to {ftp_server}.")

    # Determine folder based on year
    if year <= 2005:
        folder_path = f"{base_path}/{model}/vic_output.historical.netcdf"
    else:
        folder_path = f"{base_path}/{model}/vic_output.{scenario}.netcdf"

    # File name for the specific year
    file_name = f"{variable}.{year}.v0.nc"
    remote_file_path = f"{folder_path}/{file_name}"

    # Local file path
    local_file_path = os.path.join(local_destination, file_name)

    try:
        # Attempt to download the file
        print(f"Downloading {file_name} from {remote_file_path}...")
        with open(local_file_path, 'wb') as local_file:
            ftp.retrbinary(f"RETR {remote_file_path}", local_file.write)
        print(f"Successfully downloaded {file_name}.")
    except ftplib.error_perm as e:
        print(f"Error: File {file_name} not found on the server. ({e})")
        local_file_path = None
    finally:
        # Close the FTP connection
        ftp.quit()
        print("FTP connection closed.")

    return local_file_path

In [36]:
def process_year(model, year, variable, scenario, output_folder, gcp_bucket, ee_path):
    # Step 1: Download .nc File
    nc_file_path = download_model_files(model, year, variable, scenario, output_folder)

    if not nc_file_path:
        print(f"No .nc file for {year}. Skipping...")
        return

    # Step 2: Convert .nc to .tif
    ds = xr.open_dataset(nc_file_path)
    et = ds[variable]
    monthly_avg = et.resample(Time="1MS").mean(dim="Time")
    monthly_range = monthly_avg.max(dim="Time") - monthly_avg.min(dim="Time")

    # Handle lat/lon
    lat, lon = ds["Lat"].values, ds["Lon"].values
    lon2d, lat2d = np.meshgrid(lon, lat)
    lon_interp = np.linspace(lon2d.min(), lon2d.max(), monthly_range.shape[1])
    lat_interp = np.linspace(lat2d.min(), lat2d.max(), monthly_range.shape[0])
    lon_mesh, lat_mesh = np.meshgrid(lon_interp, lat_interp)
    lat_lon_points = np.array([lat2d.ravel(), lon2d.ravel()]).T
    monthly_range_flat = monthly_range.values.ravel()
    grid_data = griddata(lat_lon_points, monthly_range_flat, (lat_mesh, lon_mesh), method="linear")
    grid_data_flipped = np.flipud(np.where(np.isnan(grid_data), -9999, grid_data))

    transform = from_bounds(
        lon_interp.min(), lat_interp.min(), lon_interp.max(), lat_interp.max(),
        grid_data_flipped.shape[1], grid_data_flipped.shape[0]
    )
    tif_file = os.path.join(output_folder, f"{variable}_monthly_range_{year}.tif")
    with rasterio.open(
        tif_file, "w", driver="GTiff", height=grid_data_flipped.shape[0],
        width=grid_data_flipped.shape[1], count=1, dtype=grid_data_flipped.dtype,
        crs="EPSG:4326", transform=transform, nodata=-9999
    ) as dst:
        dst.write(grid_data_flipped, 1)
    print(f"GeoTIFF saved to {tif_file}")
""" 
    # Delete .nc File
    os.remove(nc_file_path)
    print(f"Deleted .nc file: {nc_file_path}")

    # Step 3: Save as COG
    cog_file = tif_file.replace(".tif", "_cog.tif")
    cog_cmd = f'gdal_translate {tif_file} {cog_file} -co TILED=YES -co COPY_SRC_OVERVIEWS=YES -co COMPRESS=LZW'
    subprocess.run(cog_cmd, shell=True)
    os.remove(tif_file)
    print(f"COG created and original GeoTIFF deleted: {cog_file}")

    # Step 4: Upload to GCP
    cp_cmd = f"gsutil cp {cog_file} gs://{gcp_bucket}/{os.path.basename(cog_file)}"
    subprocess.run(cp_cmd, shell=True)
    print(f"Uploaded COG to GCP: {cog_file}")

    gs_uri = f"gs://{gcp_bucket}/{os.path.basename(cog_file)}"
    asset_id = f"{ee_path}/vic_{variable}_range/{variable}_range_{year}".lower()


    # Step 5: Register Image in Collection
    ee_image = ee.Image.loadGeoTIFF(gs_uri)
    ee_image = ee_image.set(
            "system:time_start", ee.Date(f"{year}-01-01").millis(),
            "system:time_end", ee.Date(f"{year}-12-31").millis(),
            "model", model,
            "variable", variable,
            "scenario", scenario
        )
    task = ee.batch.Export.image.toAsset(
        image=ee_image,
        description=f"{variable}_{year}",
        assetId=asset_id,
        region=ee.Geometry.Rectangle(
            [lon_interp.min(), lat_interp.min(), lon_interp.max(), lat_interp.max()]
        ),
        scale=5000,
        maxPixels=1e13,
    )
    task.start()
    print(f"Earth Engine registration started for: {asset_id}")

    # Clean up
    os.remove(cog_file)
    print(f"Deleted local COG: {cog_file}") """


# Evapotranspiration - Actual (mm/day)

*   List item
*   List item



In [33]:
# Create the collection once
# Make sure the Image Collection ID matches your function calls
image_collection_id = "projects/pyregence-ee/assets/wrf-data/vic_et_range"
ee.data.createAsset({"type": "IMAGE_COLLECTION"}, image_collection_id)


{'type': 'IMAGE_COLLECTION',
 'name': 'projects/pyregence-ee/assets/wrf-data/vic_et_range',
 'id': 'projects/pyregence-ee/assets/wrf-data/vic_et_range',
 'updateTime': '2024-12-18T19:20:19.060869Z'}

In [38]:
# Configuration
output_folder = "./output"
gcp_bucket = "cmip5-vic"
ee_path = "projects/pyregence-ee/assets/wrf-data"
start_year, end_year = 1990, 2030
model = "CanESM2"
variable = "ET"
scenario = "rcp85"



# Process each year
for year in range(start_year, end_year + 1):
    process_year(model, year, variable, scenario, output_folder, gcp_bucket, ee_path)

Connected to gdo-dcp.ucllnl.org.
Successfully downloaded ET.1990.v0.nc.
FTP connection closed.
GeoTIFF saved to ./output/ET_monthly_range_1990.tif
Deleted .nc file: ./output/ET.1990.v0.nc
COG created and original GeoTIFF deleted: ./output/ET_monthly_range_1990_cog.tif
Uploaded COG to GCP: ./output/ET_monthly_range_1990_cog.tif
Earth Engine registration started for: projects/pyregence-ee/assets/wrf-data/vic_et_range/et_range_1990
Deleted local COG: ./output/ET_monthly_range_1990_cog.tif
Connected to gdo-dcp.ucllnl.org.
Successfully downloaded ET.1991.v0.nc.
FTP connection closed.
GeoTIFF saved to ./output/ET_monthly_range_1991.tif
Deleted .nc file: ./output/ET.1991.v0.nc
COG created and original GeoTIFF deleted: ./output/ET_monthly_range_1991_cog.tif
Uploaded COG to GCP: ./output/ET_monthly_range_1991_cog.tif
Earth Engine registration started for: projects/pyregence-ee/assets/wrf-data/vic_et_range/et_range_1991
Deleted local COG: ./output/ET_monthly_range_1991_cog.tif
Connected to gdo-d

# Evapotranspiration - Potential, natural veg (mm/day)

In [39]:
# Create the collection once
# Make sure the Image Collection ID matches your function calls
image_collection_id = "projects/pyregence-ee/assets/wrf-data/vic_pet_natveg_range"
ee.data.createAsset({"type": "IMAGE_COLLECTION"}, image_collection_id)

{'type': 'IMAGE_COLLECTION',
 'name': 'projects/pyregence-ee/assets/wrf-data/vic_pet_natveg_range',
 'id': 'projects/pyregence-ee/assets/wrf-data/vic_pet_natveg_range',
 'updateTime': '2024-12-18T20:01:57.698368Z'}

In [40]:
# Configuration
output_folder = "./output"
gcp_bucket = "cmip5-vic"
ee_path = "projects/pyregence-ee/assets/wrf-data"
start_year, end_year = 1990, 2030
model = "CanESM2"
variable = "pet_natveg"
scenario = "rcp85"



# Process each year
for year in range(start_year, end_year + 1):
    process_year(model, year, variable, scenario, output_folder, gcp_bucket, ee_path)

Connected to gdo-dcp.ucllnl.org.
Successfully downloaded pet_natveg.1990.v0.nc.
FTP connection closed.
GeoTIFF saved to ./output/pet_natveg_monthly_range_1990.tif
Deleted .nc file: ./output/pet_natveg.1990.v0.nc
COG created and original GeoTIFF deleted: ./output/pet_natveg_monthly_range_1990_cog.tif
Uploaded COG to GCP: ./output/pet_natveg_monthly_range_1990_cog.tif
Earth Engine registration started for: projects/pyregence-ee/assets/wrf-data/vic_pet_natveg_range/pet_natveg_range_1990
Deleted local COG: ./output/pet_natveg_monthly_range_1990_cog.tif
Connected to gdo-dcp.ucllnl.org.
Successfully downloaded pet_natveg.1991.v0.nc.
FTP connection closed.
GeoTIFF saved to ./output/pet_natveg_monthly_range_1991.tif
Deleted .nc file: ./output/pet_natveg.1991.v0.nc
COG created and original GeoTIFF deleted: ./output/pet_natveg_monthly_range_1991_cog.tif
Uploaded COG to GCP: ./output/pet_natveg_monthly_range_1991_cog.tif
Earth Engine registration started for: projects/pyregence-ee/assets/wrf-data